# Parsing LLM Output
This notebook takes output from finetuned Llama 65B and parses it to prepare a dataset that will be used for RLHF. 

## Load in output and parse it
This block loads in the output, parses it for solution and question, and executes the solution block. It inlcudes Try and Except blocks for cases when the model creates an incomplete output.

In [80]:
file_path = 'testing_output.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Question:")
    del sections[0]  # Remove the initial part before the first question

    for section in sections:
        try:
            question = section.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            solution = solution.split('return result')[0]
            solution = solution + '\n    return result'
            solution_text = solution
            try:
                # Execute the Python code
                exec(solution)
                answer = solution()
                data.append({"question": question, "solution": solution_text, 'answer': answer})
            except: 
                try: #pick up questions that did not indent the code block
                    solution_new = ""
                    lines = solution.split('\n')
                    indent = False
                    for line in lines:
                        if line.startswith('def solution():') or line.startswith('    return result'):
                            solution_new += line + '\n'

                        indent= True
                        if indent and not line.startswith('def solution():') and not line.startswith('    return result'):
                            solution_new += '    ' + line + '\n'
                    solution_text = solution_new
                    # Execute the Python code
                    exec(solution_new)
                    answer = solution()
                    data.append({"question": question, "solution": solution_text, 'answer': answer})
                
                except: 
                    pass
                    #pass if code doesn't execute
        except IndexError:
            pass
            #pass if there is no solution

## Export as JSON

In [82]:
import json
data_path = 'app/questions.json'

# Save data as a JSON file
with open(data_path, 'a') as json_file:
    json.dump(data, json_file, indent=4)

print("Data saved as JSON successfully.")

Data saved as JSON successfully.
